In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from art.attacks.evasion import ElasticNet
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np
import pandas as pd

from sklearn import metrics
from warnings import simplefilter

import joblib

2024-02-20 19:18:25.375281: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-20 19:18:25.406185: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-20 19:18:25.406215: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-20 19:18:25.407323: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-20 19:18:25.412794: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-20 19:18:25.413481: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test.reshape(-1), y_pred_classes, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [3]:
x_train = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/x_train.csv', low_memory=False)
y_train = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/y_train.csv', low_memory=False)
x_test = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/x_test.csv', low_memory=False)
y_test = pd.read_csv('/home/jovyan/UNSW/preprocessed_data/y_test.csv', low_memory=False)

x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()

In [4]:
model = joblib.load("/home/jovyan/UNSW/dl/new_dl/dnn.joblib")

2024-02-20 19:18:29.996427: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-02-20 19:18:30.169898: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://f9e2cd7f5b00403cb28e8a2993ea53f6: INVALID_ARGUMENT: ram://f9e2cd7f5b00403cb28e8a2993ea53f6 is a directory.


In [5]:
np.unique(y_test)

array([0, 1])

In [6]:
x_test_trimmed = x_test[47700:48000]
y_test_trimmed = y_test[47700:48000]

In [7]:
np.unique(y_test_trimmed)

array([0, 1])

In [8]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=2,
    input_shape=(56,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [9]:
# Initialize the Elastic Net attack
attack = ElasticNet(classifier=classifier, learning_rate=0.001)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test_trimmed)

EAD:   0%|          | 0/300 [00:00<?, ?it/s]

EAD: 100%|██████████| 300/300 [53:08<00:00, 10.63s/it]


In [10]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 898us/step
Accuracy: 0.17666666666666667
AUC: 0.1255924170616114

macro
Precision: 0.18661971830985916
Recall: 0.12559241706161137
F1 Score: 0.1501416430594901

weighted
Precision: 0.2625117370892019
Recall: 0.17666666666666667
F1 Score: 0.21119924457034941

Mean FNR: 0.8744075829383886
Mean TNR: 0.12559241706161137
Mean FPR: 0.8744075829383886
Mean TPR: 0.12559241706161137


In [12]:
# Initialize the Elastic Net attack
attack = ElasticNet(classifier=classifier, learning_rate=0.01)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test_trimmed)

EAD: 100%|██████████| 300/300 [53:23<00:00, 10.68s/it]


In [13]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 818us/step
Accuracy: 0.17666666666666667
AUC: 0.1255924170616114

macro
Precision: 0.18661971830985916
Recall: 0.12559241706161137
F1 Score: 0.1501416430594901

weighted
Precision: 0.2625117370892019
Recall: 0.17666666666666667
F1 Score: 0.21119924457034941

Mean FNR: 0.8744075829383886
Mean TNR: 0.12559241706161137
Mean FPR: 0.8744075829383886
Mean TPR: 0.12559241706161137


In [14]:
# Initialize the Elastic Net attack
attack = ElasticNet(classifier=classifier, learning_rate=0.1)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test_trimmed)

EAD: 100%|██████████| 300/300 [53:33<00:00, 10.71s/it]


In [15]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

10/10 [==============================] - 0s 778us/step
Accuracy: 0.17666666666666667
AUC: 0.1255924170616114

macro
Precision: 0.18661971830985916
Recall: 0.12559241706161137
F1 Score: 0.1501416430594901

weighted
Precision: 0.2625117370892019
Recall: 0.17666666666666667
F1 Score: 0.21119924457034941

Mean FNR: 0.8744075829383886
Mean TNR: 0.12559241706161137
Mean FPR: 0.8744075829383886
Mean TPR: 0.12559241706161137
